In [1]:
from requests import get
from bs4 import BeautifulSoup
import json
import time
import os
import random
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [30]:
# URL = "https://providence.craigslist.org/"
URL = "https://dallas.craigslist.org/"
QUERY = "search/apa"

DTYPE= 'craigslist-v1'

## Providence only
#postal_codes = ['02903', '02904', '02905', '02906', '02907', '02908', '02909', '02910', '02911', '02919', ]

# postal_codes = {
#     "02801": "Adamsville, Newport County",
#     "02802": "Albion, Providence County",
#     "02804": "Ashaway, Washington County",
#     "02806": "Barrington, Bristol County",
#     "02807": "Block Island, New ShorehamWashington County",
#     "02808": "Bradford, Washington County",
#     "02809": "Bristol, Bristol Count",
#     "02812": "Carolina, RichmondWashington County",
#     "02813": "Charlestown, Washington County",
#     "02814": "Chepachet, Providence County",
#     "02815": "Clayville, Providence County",
#     "02816": "Coventry, Kent County",
#     "02817": "West Greenwich, W GreenwichKent County",
#     "02818": "East Greenwich, E GreenwichKent County",
#     "02822": "Exeter, EscoheagWashington County",
#     "02823": "Fiskeville, Providence County",
#     "02824": "Forestdale, Providence County",
#     "02825": "Foster, Providence County",
#     "02826": "Glendale, Providence County",
#     "02827": "Greene, CoventryKent County",
#     "02828": "Greenville, Providence County",
#     "02829": "Harmony, Providence County",
#     "02830": "Harrisville, Burrillville, Providence County",
#     "02831": "HopeProvidence County",
#     "02832": "Hope Valley, Richmond, Washington County",
#     "02833": "Hopkinton, Washington County",
#     "02835": "Jamestown, Newport County",
#     "02836": "Kenyon, Richmond, Washington County",
#     "02837": "Little Compton, L Compton, Newport County",
#     "02838": "Manville, Providence County",
#     "02839": "Mapleville, Providence County",
#     "02840": "Newport, Newport County",
#     "02841": "Newport, Newport County",
#     "02842": "Middletown, Newport County",
#     "02852": "North Kingstown, N Kingstown, Washington County",
#     "02857": "North Scituate, N Scituate, Scituate, Providence County",
#     "02858": "Oakland, Providence County",
#     "02859": "Pascoag, Providence County",
#     "02860": "Pawtucket, Providence County",
#     "02861": "Pawtucket, Providence County",
#     "02862": "Pawtucket, Providence County",
#     "02863": "Central Falls, Providence County",
#     "02864": "Cumberland, Providence County",
#     "02865": "Lincoln, Providence County",
#     "02871": "Portsmouth, Newport County",
#     "02872": "Prudence Island, Prudence Isl, Newport County",
#     "02873": "Rockville, Washington County",
#     "02874": "Saunderstown, Washington County",
#     "02875": "Shannock, Richmond, Washington County",
#     "02876": "Slatersville, Providence County",
#     "02877": "Slocum, Washington County",
#     "02878": "Tiverton, Newport County",
#     "02879": "Wakefield, Narragansett, Peace Dale, S Kingstown, S...Washington County",
#     "02880": "Wakefield, Washington County",
#     "02881": "Kingston, Washington County",
#     "02882": "Narragansett, Point Judith, Washington County",
#     "02883": "Peace Dale, S Kingstown, South Kingstown, Washington County",
#     "02885": "Warren, Bristol County",
#     "02886": "Warwick,Kent County",
#     "02887": "Warwick,Kent County",
#     "02888": "Warwick,Kent County",
#     "02889": "Warwick,Kent County",
#     "02891": "Westerly, Washington County",
#     "02892": "West Kingston, Richmond, Washington County",
#     "02893": "West Warwick, Kent County",
#     "02894": "Wood River Junction, Wood River Jt, Washington County",
#     "02895": "Woonsocket, Providence County",
#     "02896": "North Smithfield, N Smithfield, Providence County",
#     "02898": "Wyoming, Richmond, Washington County",
#     "02901": "Providence,Providence County",
#     "02902": "Providence,Providence County",
#     "02903": "Providence,Providence County",
#     "02904": "Providence, N Providence, North Providence,Providence County",
#     "02905": "Providence, Cranston,Providence County",
#     "02906": "Providence,Providence County",
#     "02907": "Providence, Cranston,Providence County",
#     "02908": "Providence, N Providence, North Providence,Providence County",
#     "02909": "Providence,Providence County",
#     "02910": "Cranston, Providence,Providence County",
#     "02911": "North Providence, N Providence, Providence,Providence County",
#     "02912": "Providence, Brown Station, Providence County",
#     "02914": "East Providence, E Providence,Providence County",
#     "02915": "Riverside,Providence County",
#     "02916": "Rumford,Providence County",
#     "02917": "Smithfield,Providence County",
#     "02918": "Providence, Friar Station,Providence County",
#     "02919": "Johnston, Providence,Providence County",
#     "02920": "Cranston, Providence County",
#     "02921": "Cranston, Providence County",
#     "02940": "Providence, Providence County",
# }

postal_codes = {
    "75023": "Plano",
    "75024": "Plano",
    "75025": "Plano",
    "75040": "Garland",
    "75041": "Garland",
    "75042": "Garland",
    "75043": "Garland",
    "75044": "Garland",
    "75048": "Garland",
    "75074": "Plano",
    "75075": "Plano",
    "75080": "Richardson",
    "75081": "Richardson",
    "75082": "Richardson",
    "75093": "Plano",
    "75094": "Plano",
    "75240": "Far North Dallas",
    "75248": "Far North Dallas",
    "75252": "Far North Dallas",
    "75254": "Far North Dallas",
    "75287": "Far North Dallas",
}


def query_url():
    return URL + QUERY

def filter_line(line):
    if not line or line == 'QR Code Link to This Post':
        return False
    else:
        return True

def map_line(line):
    return line.strip('\n').strip(' ')

def get_description(href):
    time.sleep(random.uniform(0.2, 0.8))
    response_details = get(href)
    detail_soup = BeautifulSoup(response_details.text, 'html.parser')
    body = detail_soup.find(id="postingbody")
    
    return list(filter(filter_line, map(map_line, body.find_all(text=True))))

def extract_repost(duplicate_rows):
    if duplicate_rows:
        return [row.get('data-repost-of') for row in duplicate_rows.find_all(class_="result-row")]
    else:
        return []

def listing(post):
    title = post.find(class_ = "result-title hdrlnk")
    print("Getting: {}".format(title.text))
    return {
        'price': post.find(class_ = "result-price").text, 
        'title': title.text,
        'href': title.get('href'),
        'description': get_description(title.get('href')),
        'duplicates': extract_repost(post.find(class_="duplicate-rows")),
    }


def extract_listings(postal_code, area):
    response = get(
        query_url(), 
        params={
            'bundleDuplicates':1, 
            'availabilityMode':0, 
            'sale_date':'all dates', 
            'search_distance': 0, 
            'postal': postal_code
        }
    )
    html_soup = BeautifulSoup(response.text, 'html.parser')
    posts = html_soup.find_all('li', class_= ['result-row', 'result-row duplicate-row'])
    dup_posts = html_soup.find_all('li', class_= 'result-row duplicate-row')

    print(
        "Got type: {}; listings: {}; dup listings: {} for {}".format(
            type(posts), len(posts), len(dup_posts), postal_code)
    )

    # Is there pagination? Seems to match UI well but not exactly?
    extracted = [listing(post) for post in posts + dup_posts]
   
    
    raw_path = os.path.join(DTYPE, 'raw_html')
    os.makedirs(raw_path, exist_ok=True)
    with open(os.path.join(raw_path, '{}-{}.html'.format(postal_code, area)), 'w', encoding='utf-8') as f:
        f.write(response.text)

    extracted_path = os.path.join(DTYPE, 'extracted')
    os.makedirs(extracted_path, exist_ok=True)
    with open(os.path.join(extracted_path, '{}-{}.json'.format(postal_code, area)), 'w', encoding='utf-8') as f:
        json.dump(extracted, f, indent=2)
        
# extract_listings('02860', 'foo_bar, baz')
        
for postal_code, area in postal_codes.items():
     extract_listings(postal_code, area)
        

Got type: <class 'bs4.element.ResultSet'>; listings: 134; dup listings: 11 for 75023
Getting: Amazing 3/2.5/2!!! Ready for Immediate Move IN!
Getting: YOU WOULDN'T BELIEVE US, IF WE TOLD YOU
Getting: Great 2 bedrooms, with upgrades!
Getting: 1 Month Free! Call For More Details!
Getting: GE Stainless Steel Appliances, Hardwood Plank Flooring, Carports
Getting: Experience A More Alluring Lifestyle!!
Getting: UNIQUE SPACES DESIGNED FOR AN URBAN LIFESTYLE
Getting: UNIQUE SPACES DESIGNED FOR AN URBAN LIFESTYLE
Getting: AMAZING AVAILABLE IMMEDIATELY!
Getting: THIS WILL GO FAST SO HURRY IN
Getting: YOUR NEXT APARTMENT IS READY
Getting: 24-Hour Fitness Center, Full-Size Washer / Dryer Connections, Carports
Getting: YOU WOULDN'T BELIEVE US, IF WE TOLD YOU
Getting: Experience next-level apartment home features and community amenities
Getting: 6wks free  Low Credit - Bad Credit - Property Debt - Broken Lease NO P
Getting: 1 Month Free! Call For More Details!
Getting: MAY WE HAVE YOUR ATTENTION PL

In [39]:
def load_listings(postal_code, area):
    extracted_path = os.path.join(DTYPE, 'extracted')
    with open(os.path.join(extracted_path, '{}-{}.json'.format(postal_code, area)), 'r', encoding='utf-8') as f:
        return json.load(f)

zip_codes = {}
for postal_code, area in postal_codes.items():
    name = "{}_{}".format(postal_code, area)
    zip_codes[name] = load_listings(postal_code, area)

In [47]:
zip_codes['75023_Plano']

[{'price': '$1995',
  'title': 'Amazing 3/2.5/2!!! Ready for Immediate Move IN!',
  'href': 'https://dallas.craigslist.org/dal/apa/d/plano-amazing-ready-for-immediate-move/7124101182.html',
  'description': ['This amazing property is ready for immediate move in. The home has a very spacious and functional floor plan with multiple living areas. Downstairs consists of family room, formal living, and formal dining along with kitchen, laundry, breakfast nook as well as half bath. The upstairs has all three bedrooms, secondary bath, and nice loft area which would be great for home office, game room, etc. Located inside Plano ISD, only minutes from Central Expressway, and surrounded by shops, restaurants, and entertainment this property is sure to lease quickly.',
   'Location',
   'Address: 904 Gordon Oaks Drive',
   'City: Plano',
   'State: TX',
   'Zip Code: 75023',
   'County: Collin',
   'Subdivison: Creek Trails',
   'Interior Information',
   'Square Feet: 2068',
   'Bedrooms: 3',
  

In [49]:
[listing['price'] for listing in zip_codes['75023_Plano']]

['$1995',
 '$1367',
 '$968',
 '$1400',
 '$1388',
 '$890',
 '$1037',
 '$1171',
 '$931',
 '$1355',
 '$2350',
 '$1119',
 '$1696',
 '$1625',
 '$1291',
 '$1825',
 '$1215',
 '$1340',
 '$1195',
 '$1300',
 '$1419',
 '$1290',
 '$1520',
 '$1045',
 '$1359',
 '$1359',
 '$1550',
 '$1305',
 '$1305',
 '$890',
 '$1825',
 '$1197',
 '$1852',
 '$1219',
 '$1215',
 '$1825',
 '$1215',
 '$1398',
 '$1',
 '$1945',
 '$1318',
 '$1318',
 '$1550',
 '$962',
 '$1525',
 '$1460',
 '$1460',
 '$1290',
 '$1290',
 '$1290',
 '$1290',
 '$1290',
 '$1125',
 '$1125',
 '$1125',
 '$2010',
 '$1183',
 '$1',
 '$1875',
 '$1190',
 '$1550',
 '$1550',
 '$1550',
 '$1550',
 '$1550',
 '$1295',
 '$1495',
 '$1022',
 '$1195',
 '$1300',
 '$1370',
 '$1290',
 '$1430',
 '$1291',
 '$1291',
 '$1235',
 '$1',
 '$921',
 '$1950',
 '$1355',
 '$1355',
 '$967',
 '$896',
 '$1398',
 '$1398',
 '$1059',
 '$1',
 '$1',
 '$1',
 '$1',
 '$1635',
 '$1355',
 '$1525',
 '$1195',
 '$1460',
 '$1190',
 '$1430',
 '$1460',
 '$1340',
 '$1300',
 '$1367',
 '$1290',
 '$1290',

In [139]:
match = 'voucher'
l = [
    [
        "\n".join([listing['title'], listing['price'], listing['href']] + listing['description']) for listing in zipcode 
        if match in " ".join(listing['description']).lower() and int(listing['price'][1:]) > 1
    ] for zipcode in zip_codes.values()
]

for item in [item for sublist in l for item in sublist]:
    print(item)
    print()

3515 Hilltop Lane
$1950
https://dallas.craigslist.org/ndf/apa/d/plano-3515-hilltop-lane/7111313483.html
Freshly updated home for lease in Plano. Home has just been updated with new carpet and fresh paint throughout. Vast living spaces with amply sized in 3 bedrooms. Split bedroom plan. Master bathroom boasts a tub and a separate shower. Fantastic find in excellent location in Plano. Washer & dryer connections available and 2 car garage. Applications can be completed and paid online through our website. If Active it is still available. Not eligible for housing vouchers. Tenants will be required to carry renter's insurance during the term of the lease. Text listing agent to request any additional information or to schedule a showing. Tenants must bring their own refrigerator, washer and dryer. No Pets.

2 YEAR OLD EVICTION OR A 1 YEAR OLD BROKEN LEASE ? I CAN HELP!!!!
$695
https://dallas.craigslist.org/ndf/apa/d/dallas-2-year-old-eviction-or-1-year/7117506581.html
My name is STEPHEN TEWE